In [4]:
from library.load_split.load_split import Load_Split
from library.features.features import Polynomial, Standardize, One_Hot_Enc
from library.model.model import Model
from library.preprocessor.preprocessor import PreProcessor
from library.metrics.metrics import Metrics
from sklearn.metrics import mean_squared_error

In [5]:
#Loading Data
df = Load_Split('flights_data.csv')

subset = ['destination_city', 'source_city', 'class']

# Cleaning Data for Train
train = PreProcessor(df.train)
train.clean(subset)
train = train.clean

# Cleaning Data for Test
test = PreProcessor(df.test)
test.clean(subset)
test = test.clean



# Fill NA with mean for Train Data
train = PreProcessor(train)
train.mean_na(train.df[subset])
train = train.fill


# Fill NA with mean for Test Data
test = PreProcessor(test)
test.mean_na(test.df[subset])
test = test.fill

# Train Data Features 

# We don't want to standardize 'price' so we save it before dropping it from our Standardization
y_train = train['price']
y_test = test['price']

train = train.drop('price', axis = 1)
train = Standardize(train.select_dtypes(exclude = ['object']))
train = train.transform()

train = One_Hot_Enc(train.select_dtypes(exclude = ['object']))
train = train.transform()

train = Polynomial(train.select_dtypes(exclude = ['object']))
train = train.transform(2)

# Test Data Features 

test = test.drop('price', axis = 1)
test = Standardize(test.select_dtypes(exclude =['object']))
test = test.transform()

test = One_Hot_Enc(test.select_dtypes(exclude = ['object']))
test = test.transform()

test = Polynomial(test.select_dtypes(exclude =['object']))
test = test.transform(2)

# Declaring our X matrixes 
X_test = test
X_train = train

# Training the model
model = Model(X_train, y_train)
model.train(reg = "l2")
y_pred_train = model.predict(X_train)
prediction = model.predict(X_test)
test['predictions'] = prediction


# Scoring 

print('Test Score')
score = Metrics.mse(y_test, test['predictions'])

score


/Users/luisquinonespr/code/CDS_Final/library/preprocessor/preprocessor.py:11: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  self.fill = self.df.fillna(self.df.mean())


In [ ]:
test




,1,Unnamed: 0,duration,days_left,Unnamed: 0^2,Unnamed: 0 duration,Unnamed: 0 days_left,duration^2,duration days_left,days_left^2,predictions
0,1.0,0.203704,0.501963,-0.149230,0.041495,0.102252,-0.030399,0.251967,-0.074908,0.022270,2271.02
1,1.0,0.701340,2.078687,1.032409,0.491877,1.457866,0.724069,4.320939,2.146054,1.065868,8739.00
2,1.0,0.570587,1.336208,-0.444639,0.325570,0.762424,-0.253706,1.785453,-0.594131,0.197704,2055.11
3,1.0,-0.893175,0.004196,-0.518492,0.797762,-0.003748,0.463104,0.000018,-0.002176,0.268834,1033.12
4,1.0,-1.236879,-0.957967,1.697080,1.529869,1.184889,-2.099083,0.917701,-1.625747,2.880081,892.26
...,...,...,...,...,...,...,...,...,...,...,...
150072,1.0,-1.335358,1.325085,1.253966,1.783182,-1.769463,-1.674494,1.755850,1.661611,1.572430,1871.50
150073,1.0,0.471785,-0.412927,0.220032,0.222581,-0.194813,0.103808,0.170508,-0.090857,0.048414,136.12
150074,1.0,1.579440,0.501963,0.884704,2.494632,0.792820,1.397337,0.251967,0.444088,0.782701,8487.22
150075,1.0,1.332509,-1.027488,-0.001525,1.775580,-1.369136,-0.002032,1.055731,0.001567,0.000002,7992.67


In [ ]:
y_test

167841     6461
210970    59033
199638     6147
72777      4593
42989      5622
          ...  
34454      5950
191075     1830
287073    51595
265672    31982
200941     2622
Name: price, Length: 150077, dtype: int64

674778018.9880183